In [46]:
import numpy as np
import open3d as o3d
import os
from show import show_pointcloud

class intrinsics():
    def __init__(self,fx,fy,ppx,ppy):
        self.fx = fx
        self.fy = fy
        self.ppx = ppx
        self.ppy = ppy
        

date = "2024-12-27"
time = "20-04"
folders = ["depth","rgb"]


class RealSenseImages():
    def __init__(self):
        self.intrinsics = intrinsics(0,0,0,0)
        self.raw = {}
        self.depth_image = []
        self.color_image = []
        self.pcd = None
        
    def set_intrinsics(self,i):
        self.intrinsics = i
    
    def load_images(self,date,time):
        folders = ["depth","rgb"]
        for folder in folders:
            self.raw[folder] = []
            folder_path = os.path.join("data",date,time,folder)
            for i in range(10):
                image_path = os.path.join(folder_path,"%d.npy" % i)
                image = np.load(image_path)
                self.raw[folder].append(image)
                
    def calculate_pointclouds(self,save_path = "outnew.pcd"):
        o3d_depth = o3d.geometry.Image(self.depth_image.astype(np.uint16))
        o3d_color = o3d.geometry.Image(self.color_image.astype(np.uint8))
        o3d_rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(o3d_color, o3d_depth, convert_rgb_to_intensity=False)
        
        o3d_intrinsic = o3d.camera.PinholeCameraIntrinsic(
            width = self.depth_image.shape[1], 
            height = self.depth_image.shape[0], 
            fx = self.intrinsics.fx, 
            fy = self.intrinsics.fy,
            cx = self.intrinsics.ppx, 
            cy = self.intrinsics.ppy)

        self.pcd = o3d.geometry.PointCloud.create_from_rgbd_image(o3d_rgbd, o3d_intrinsic)
        o3d.io.write_point_cloud("outnew.pcd", self.pcd)
        
    
    def fliter_images(self):
        height, width = self.raw["depth"][0].shape
        self.depth_image = np.zeros((height,width))
        self.color_image = np.zeros((height,width,3))
        depth_list = np.stack(self.raw["depth"], axis=0)
        median_depth = np.median(depth_list, axis=0).astype(np.uint16)
        max_depth = np.max(depth_list, axis=0).astype(np.uint16)
        min_depth = np.min(depth_list, axis=0).astype(np.uint16)
        median_color = np.median(np.stack(self.raw["rgb"],axis=0),axis=0).astype(np.uint8)
        for y in range(height):
            for x in range(width):
                if max_depth[y,x] - min_depth[y,x] < median_depth[y,x]:
                    self.depth_image[y,x] = median_depth[y,x]
                    self.color_image[y,x] = median_color[y,x]

In [47]:
depth_intrinsics = intrinsics(652.8689575195312,652.8689575195312,651.2142944335938,363.8994140625)
rsi = RealSenseImages()
rsi.set_intrinsics(depth_intrinsics)
rsi.load_images("2024-12-27","20-04")

In [48]:
rsi.fliter_images()

In [49]:
rsi.calculate_pointclouds()